In [1]:
!pip install -q torch \
    transformers \
    accelerate \
    bitsandbytes \
    langchain \
    sentence-transformers \
    faiss-gpu \
    openpyxl \
    pacmap \
    python-dotenv \
    pandas \
    datasets \
    matplotlib \
    ragatouille \
    langchain-openai \
    ragas

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 24.6.1 requires cubinlinker, which is not installed.
cudf 24.6.1 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.6.1 requires ptxcompiler, which is not installed.
cuml 24.6.1 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 24.6.1 requires cupy-cuda11x>=12.0.0, which is not installed.
keras-cv 0.9.0 requires keras-core, which is not installed.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 16.1.0 which is incompatible.
cudf 24.6.1 requires cuda-python<12.0a0,>=11.7.1, but you have cu

In [2]:
from tqdm.notebook import tqdm
import pandas as pd
from typing import Optional, List, Tuple
from datasets import Dataset
import matplotlib.pyplot as plt
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import datasets
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy
from datetime import datetime
import ragas
from ragatouille import RAGPretrainedModel

pd.set_option(
    "display.max_colwidth", None
)  # this will be helpful when visualizing retriever outputs

In [3]:
import torch
torch.cuda.device_count()

2

In [331]:
QUESTION = 'I have a scenario for a disorder can say what is the sub type based on senario? the senario is: A 30-year-old man began using prescription painkillers for a back injury and now uses opioids daily, unable to stop despite severe health and legal issues.'

# Embedding model for RAG first retrieval
ENCODER = "thenlper/gte-small"

# Config for the optional reranking step
NUM_RETRIEVED_DOCS = 15
RERANKER = 'colbert-ir/colbertv2.0'
NUM_RERANKED_DOCS = 10

# LLM = 'gpt-3.5-turbo-0125'   "microsoft/Phi-3-mini-128k-instruct"

In [71]:
import re

def clean_text(text):
    # Remove headers, footers, 
    text = remove_headers_and_footers(text)
    text = remove_copyright_info(text)
    text = remove_redundant_section_headers(text)


    text = re.sub(r'\n+', '\n', text)  # Normalize multiple newlines
    text = re.sub(r'\s+', ' ', text)  # Remove excessive whitespace
    text = re.sub(r'[^\x00-\x7F]+', '', text)  # Remove non-ASCII characters

    # Remove specific patterns
    text = re.sub(r'(DSM-5TM)', '', text, flags=re.IGNORECASE)
    text = re.sub(r'(American Psychiatric Association)', '', text, flags=re.IGNORECASE)

    return text.strip()

def remove_headers_and_footers(text):
    lines = text.splitlines()
    clean_lines = []
    for line in lines:
        if not line.strip().startswith('DIAGNOSTIC AND STATISTICAL MANUAL') and \
           not line.strip().startswith('DSM-5') and \
           not line.strip().startswith('American Psychiatric Association'):
            clean_lines.append(line)
    return '\n'.join(clean_lines)

def remove_copyright_info(text):
    copyright_patterns = [
        r'Copyright © [0-9]{4} American Psychiatric Association',
        r'ALL RIGHTS RESERVED',
        r'Correspondence regarding copyright permissions should be directed to DSM Permissions',
        r'Manufactured in the United States of America on acid-free paper',
        r'Library of Congress Cataloging-in-Publication Data'
    ]
    for pattern in copyright_patterns:
        text = re.sub(pattern, '', text, flags=re.IGNORECASE)
    return text

def remove_redundant_section_headers(text):
    section_patterns = [
        r'Section I[\s\S]*?Section II',
        r'Section III[\s\S]*?Appendix'
    ]
    for pattern in section_patterns:
        text = re.sub(pattern, '', text, flags=re.IGNORECASE)
    return text

# Load and clean the text
# with open('DSM5.txt', 'r') as file:
#     raw_text = file.read()

# cleaned_text = clean_text(raw_text)

# # Save the cleaned text to a new file
# with open('DSM5_cleaned.txt', 'w') as file:
#     file.write(cleaned_text)

# print("Text cleaning and preprocessing completed.")

In [292]:
with open('/kaggle/input/dataset/DSM5 ENGLISH.txt','r') as file:
  raw_text = file.read()
  text = clean_text(raw_text)
with open('/kaggle/input/dataset/final_disease_subdisease_summaries.txt','r') as file:
  text2 = file.read()
ds = [text]

In [293]:
ds2 = [text2]

In [294]:
from langchain.docstore.document import Document as LangchainDocument

LANGCHAIN_RAW_KNOWLEDGE_BASE = [
    LangchainDocument(page_content=doc)
    for doc in ds
]

In [295]:
LANGCHAIN_DISEASES_KNOWLEDGE_BASE = [
    LangchainDocument(page_content=doc)
    for doc in ds2
]

In [296]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer
import tqdm

CHUNK_SIZE = 256 # because we know it's the max seq length from the embedding model
MARKDOWN_SEPARATORS = [
    "\n#{1,6} ",
    "\n\n",
    "\n",
    " ",
    "",
]

text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    AutoTokenizer.from_pretrained(ENCODER),
    chunk_size=CHUNK_SIZE,
    chunk_overlap=int(CHUNK_SIZE / 10),
    add_start_index=True,
    strip_whitespace=True,
    separators=MARKDOWN_SEPARATORS,
)

# same as before, just with the new tokenizer
docs_processed = []
for doc in tqdm.tqdm(LANGCHAIN_RAW_KNOWLEDGE_BASE):
    docs_processed += text_splitter.split_documents([doc])


diseases_docs_processed = []
for doc in tqdm.tqdm(LANGCHAIN_DISEASES_KNOWLEDGE_BASE):
    diseases_docs_processed += text_splitter.split_documents([doc])

100%|██████████| 1/1 [00:00<00:00, 22.60it/s]


In [297]:
# Remove duplicates
unique_texts = set()
docs_processed_unique = []
for doc in docs_processed:
    if doc.page_content not in unique_texts:
        unique_texts.add(doc.page_content)
        docs_processed_unique.append(doc)

In [298]:
print(len(docs_processed))
print(len(docs_processed_unique))

5007
5005


In [299]:
# Remove duplicates
unique_texts_diseases = set()
diseases_docs_processed_unique = []
for doc in diseases_docs_processed:
    if doc.page_content not in unique_texts_diseases:
        unique_texts_diseases.add(doc.page_content)
        diseases_docs_processed_unique.append(doc)

In [300]:
print(len(diseases_docs_processed))
print(len(diseases_docs_processed_unique))

16
16


In [301]:
def get_cur_time():
  now = datetime.now()
  return now.isoformat()

embedding_model = HuggingFaceEmbeddings(
    model_name=ENCODER,
    multi_process=True,
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True},  # set True for cosine similarity
)

start = get_cur_time()
print(f'Starting embedding at {start}')
KNOWLEDGE_VECTOR_DATABASE = FAISS.from_documents(
    docs_processed_unique, embedding_model, distance_strategy=DistanceStrategy.COSINE
)

end = get_cur_time()
print(f'Finished embedding at {end}')

Starting embedding at 2024-08-16T15:45:49.564548
Finished embedding at 2024-08-16T15:46:02.136990


In [302]:
start = get_cur_time()
print(f'Starting embedding at {start}')
KNOWLEDGE_VECTOR_DATABASE_1 = FAISS.from_documents(
    diseases_docs_processed_unique, embedding_model, distance_strategy=DistanceStrategy.COSINE
)

end = get_cur_time()
print(f'Finished embedding at {end}')

Starting embedding at 2024-08-16T15:46:02.143773
Finished embedding at 2024-08-16T15:46:09.780748


In [303]:
# embed a user query in the same space
query_vector = embedding_model.embed_query(QUESTION)

In [304]:
KNOWLEDGE_VECTOR_DATABASE_1.save_local('test1')

In [305]:
KNOWLEDGE_VECTOR_DATABASE.save_local('test')

In [19]:
!pip install langchain-huggingface transformers -qU

In [306]:
from langchain_huggingface import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_id = "microsoft/Phi-3-mini-128k-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map='cuda:1', trust_remote_code = True)
pipe = pipeline(
    "text-generation", model=model, tokenizer=tokenizer, max_new_tokens=128
)
hf_pipe = HuggingFacePipeline(pipeline=pipe)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [332]:
class Reader:

    def __init__(self):#, openai_api_key: str):

        self.llm = hf_pipe#ChatOpenAI(openai_api_key=openai_api_key, model=LLM)
        #self.openai_api_key = openai_api_key
        self.output_parser = StrOutputParser()

    def base_llm_answer(self, question: str) -> str:
        # Enhance the question first
        base_prompt = ChatPromptTemplate.from_messages([
            ("system", "You are an expert assistant specialized in psychology. "
                       "Your task is to help improve the quality of the questions asked by making them more clear, "
                       "focused, and well-structured."),
            ("user", "Please refine the following question for better clarity and focus: '{question}'")
        ])
        chain = base_prompt | self.llm | self.output_parser
        enhanced_question = chain.invoke({'question': question})
        return enhanced_question

    def rag_llm_answer(self, question: str, context: Optional[str] = None) -> str:
        rag_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a licensed psychologist with extensive expertise in diagnosing psychological disorders.
    Your task is to analyze the provided scenario and determine the most likely psychological disorder based solely on the data given to you.

    Important Instructions:
    1. You must identify the disorder in this strict format:
       Disorder Name Category: Broader Category Name
       (e.g., Autism Spectrum Disorder Category: Neurodevelopmental Disorders)
    2. After identifying the disorder, provide a clear and well-reasoned explanation for why this specific disorder was diagnosed, strictly using the information from the scenario. All reasoning must be rooted in the data presented.
    3. It is forbidden to provide an answer in any format other than the specified structure.
    4. You must always provide an answer based on the data given in the scenario, even if the information seems limited or ambiguous. Stick to the data and do not introduce external assumptions.

    Ensure that your response is professional, concise, and based solely on the provided data, following the required structure without any deviations."""),
    
    ("user", """Context:
    {context}
    ---
    Now, here is the question you need to answer:

    Question: {question}""")
])
        
        chain = rag_prompt | self.llm | self.output_parser
        answer = chain.invoke({'context': context, 'question': question})
        return answer

# Let's initialize our Reader class, and use it to answer our question based on previously retrieved documents! First for a small test.

reader = Reader()#openai_api_key=openai_api_key)


In [333]:


def retrieved_docs_to_prompt_context(retrieved_docs):
    retrieved_docs_text = [
        doc.page_content for doc in retrieved_docs
    ] 
    context = "\nExtracted documents:\n"
    context += "".join(
        [f"Document {str(i)}:::\n" + doc for i, doc in enumerate(retrieved_docs_text)]
    )
    return retrieved_docs_text

def rag_pipeline(question, reranker=None, return_context=False):
   
    enhanced_question = question


    retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(
        query=enhanced_question, k=NUM_RETRIEVED_DOCS)


    prompt_context = retrieved_docs_to_prompt_context(retrieved_docs)

    if reranker:
        relevant_docs_reranked = reranker.rerank(enhanced_question,
                                                 [doc.page_content for doc in retrieved_docs],
                                                 k=NUM_RERANKED_DOCS)


        relevant_docs_reranked_text = [doc["content"] for doc in relevant_docs_reranked]

        rag_answer = reader.rag_llm_answer(enhanced_question, relevant_docs_reranked_text)
    else:
        rag_answer = reader.rag_llm_answer(enhanced_question, prompt_context)

    if return_context:
        return (rag_answer, retrieved_docs)
    else:
        return rag_answer


def classificationn_pipeline(question, reranker=None, return_context = False):
    retrieved_docs = KNOWLEDGE_VECTOR_DATABASE_1.similarity_search(
    query=question, k=1)
    print(f'=> Retrieved {len(retrieved_docs)} docs')
    prompt_context = retrieved_docs_to_prompt_context(retrieved_docs)
    if reranker:
       print(f"=> Reranking documents...")
       relevant_docs_reranked = reranker.rerank(question,
                                               [doc.page_content for doc in retrieved_docs],
                                               k=1)
       print(f'=> After reranking, {len(relevant_docs_reranked)} docs')
       relevant_docs_reranked_text = [doc["content"] for doc in relevant_docs_reranked]
       rag_answer = reader.rag_llm_answer(question, relevant_docs_reranked_text)
       if return_context:
        return (rag_answer, relevant_docs_reranked)
    else:
       rag_answer = reader.rag_llm_answer(question, prompt_context)
       if return_context:
          return (rag_answer, retrieved_docs)
       else:
          return rag_answer

In [323]:
QUESTION = 'I have a scenario for a disorder, can you say what is the sub type based on senario? Please explain. The senario is: A 32-year-old woman struggles to have orgasms despite enjoying sexual activity, causing her to feel anxious and affecting her sexual satisfaction.'

In [90]:
# no rag
print(reader.base_llm_answer(QUESTION))

System: You are an expert assistant specialized in psychology. Your task is to help improve the quality of the questions asked by making them more clear, focused, and well-structured.
Human: Please refine the following question for better clarity and focus: 'I have a scenario for a disorder, can you say what is the sub type based on senario? Please explain. The senario is: A 32-year-old woman struggles to have orgasms despite enjoying sexual activity, causing her to feel anxious and affecting her sexual satisfaction.'
Assistant: Refined Question: "What subtype of sexual dysfunction might be indicated by a 32-year-old woman's difficulty in achieving orgasm despite enjoying sexual activity, and how does this condition affect her sexual satisfaction and emotional well-being?"

Explanation: The refined question is more focused and clear, as it specifically asks for the subtype of sexual dysfunction that may be indicated by the given scenario. It also highlights the impact of the condition 

In [334]:
RERANKER_MODEL_NAME = "colbert-ir/colbertv2.0"
reranker = RAGPretrainedModel.from_pretrained(RERANKER_MODEL_NAME)
que = " A 32-year-old woman struggles to have orgasms despite enjoying sexual activity, causing her to feel anxious and affecting her sexual satisfaction"
try:
    rag_answer = rag_pipeline(que, reranker=reranker, return_context=False)
    print(rag_answer)
    print("*********************")
    start_index = rag_answer.find("Answer:")

    # If "Answer:" is found, print everything after that
    if start_index != -1:
        print(rag_answer[start_index:].strip())
    else:
        print(rag_answer)
except Exception as e:
    print(f'ERROR! {e}')


100%|██████████| 1/1 [00:00<00:00, 28.53it/s]


System: You are a licensed psychologist with extensive expertise in diagnosing psychological disorders.
    Your task is to analyze the provided scenario and determine the most likely psychological disorder based solely on the data given to you.

    Important Instructions:
    1. You must identify the disorder in this strict format:
       Disorder Name Category: Broader Category Name
       (e.g., Autism Spectrum Disorder Category: Neurodevelopmental Disorders)
    2. After identifying the disorder, provide a clear and well-reasoned explanation for why this specific disorder was diagnosed, strictly using the information from the scenario. All reasoning must be rooted in the data presented.
    3. It is forbidden to provide an answer in any format other than the specified structure.
    4. You must always provide an answer based on the data given in the scenario, even if the information seems limited or ambiguous. Stick to the data and do not introduce external assumptions.

    Ensur

In [92]:
try:
    rag_answer = classificationn_pipeline(QUESTION, reranker=False, return_context=False)
    print(rag_answer)
except Exception as e:
    print(f'ERROR! {e}')

=> Retrieved 1 docs
System: You are a professional psychologist with expertise in diagnosing mental health conditions.
            Based on the provided scenario in the context, your task is to:

            1. Identify the **most probable psychological disorder** based on the symptoms presented.
            2. State the **broader category or classification** to which this disorder belongs (e.g., Neurodevelopmental Disorders, Anxiety Disorders).
            3. Provide **reasoning** for your diagnosis based on established psychological principles and diagnostic criteria.
            4. Respond as a **licensed psychologist**, ensuring your explanation is professional and clear without any mention of being an AI or automated system.

            Focus solely on the most probable disorder and offer a clear, well-reasoned explanation.
Human: Context:
            ['Sexual Dysfunctions - Female Orgasmic Disorder : This sub-disease typically involves female orgasmic disorder\nKeywords: sexual,

In [93]:
QUESTION = 'I have a scenario for a disorder can say what is the sub type based on senario? the senario is: A 22-year-old woman isolates herself in her room, rarely speaks to anyone, and has lost interest in personal hygiene. She reports seeing shadowy figures in her home.'

In [94]:
try:
    (rag_answer, rag_context) = classificationn_pipeline(QUESTION, reranker=False, return_context=True)
    print(rag_answer)
except Exception as e:
    print(f'ERROR! {e}')

=> Retrieved 1 docs
System: You are a professional psychologist with expertise in diagnosing mental health conditions.
            Based on the provided scenario in the context, your task is to:

            1. Identify the **most probable psychological disorder** based on the symptoms presented.
            2. State the **broader category or classification** to which this disorder belongs (e.g., Neurodevelopmental Disorders, Anxiety Disorders).
            3. Provide **reasoning** for your diagnosis based on established psychological principles and diagnostic criteria.
            4. Respond as a **licensed psychologist**, ensuring your explanation is professional and clear without any mention of being an AI or automated system.

            Focus solely on the most probable disorder and offer a clear, well-reasoned explanation.
Human: Context:
            ['Personality Disorders - Exhibitionistic Disorder : This sub-disease typically involves exhibitionistic disorder\nKeywords: self-

In [336]:
que = "A 22-year-old woman isolates herself in her room, rarely speaks to anyone, and has lost interest in personal hygiene. She reports seeing shadowy figures in her home."
try:
    (rag_answer, rag_context) = rag_pipeline(que, reranker=reranker, return_context=True)
    print(rag_answer)
except Exception as e:
    print(f'ERROR! {e}')

100%|██████████| 1/1 [00:00<00:00, 33.22it/s]


System: You are a licensed psychologist with extensive expertise in diagnosing psychological disorders.
    Your task is to analyze the provided scenario and determine the most likely psychological disorder based solely on the data given to you.

    Important Instructions:
    1. You must identify the disorder in this strict format:
       Disorder Name Category: Broader Category Name
       (e.g., Autism Spectrum Disorder Category: Neurodevelopmental Disorders)
    2. After identifying the disorder, provide a clear and well-reasoned explanation for why this specific disorder was diagnosed, strictly using the information from the scenario. All reasoning must be rooted in the data presented.
    3. It is forbidden to provide an answer in any format other than the specified structure.
    4. You must always provide an answer based on the data given in the scenario, even if the information seems limited or ambiguous. Stick to the data and do not introduce external assumptions.

    Ensur

In [37]:
QUESTION = 'I have a scenario for a disorder can say what is the sub type based on senario? the senario is: An 8-year-old boy has limited interests, focusing only on trains. He struggles to interact with peers and prefers solitary activities.'

In [38]:
try:
    (rag_answer, rag_context) = rag_pipeline(QUESTION, reranker=False, return_context=True)
    print(rag_answer)
except Exception as e:
    print(f'ERROR! {e}')

=> Retrieved 4 docs

System: Using the information contained in the context,

            give a comprehensive answer to the question.

            Respond only to the question asked, response should be concise and relevant to the question.

            Provide the number of the source document when relevant.

            If the answer cannot be deduced from the context, do not give an answer.

Human: Context:

            ['Subtypes\nThree subtypes of conduct disorder are provided based on the age at onset of the disorder. Onset is most accurately estimated with information from both the youth and the care\xad giver; estimates are often 2 years later than actual onset. Both subtypes can occur in a mild, moderate, or severe form. An unspecified-onset subtype is designated when there is in\xad sufficient information to determine age at onset.\n   In childhood-onset conduct disorder, individuals are usually male, frequently display physical aggression toward others, have disturbed peer r

In [39]:
try:
    (rag_answer, rag_context) = classificationn_pipeline(QUESTION, reranker=False, return_context=True)
    print(rag_answer)
except Exception as e:
    print(f'ERROR! {e}')

=> Retrieved 1 docs

System: Using the information contained in the context,

            give a comprehensive answer to the question.

            Respond only to the question asked, response should be concise and relevant to the question.

            Provide the number of the source document when relevant.

            If the answer cannot be deduced from the context, do not give an answer.

Human: Context:

            ['Neurocognitive Disorders - Mild Neurocognitive Disorder : This sub-disease typically involves mild neurocognitive disorder\nKeywords: use, judgment, disorientation, memory, opioid, confusion, cognitive decline, mild, major, attention\n\nNeurocognitive Disorders - Opioid Use Disorder : This sub-disease typically involves opioid use disorder\nKeywords: use, judgment, disorientation, memory, opioid, confusion, cognitive decline, mild, major, attention\n\nNeurodevelopmental Disorders - Attention-Deficit/Hyperactivity Disorder (ADHD) : This sub-disease typically involve

In [40]:
QUESTION = 'I have a scenario for a disorder can say what is the sub type based on senario? the senario is: An 11-year-old boy is forgetful in daily activities, frequently misplacing his belongings and failing to meet deadlines for school projects.'

In [41]:
try:
    (rag_answer, rag_context) = classificationn_pipeline(QUESTION, reranker=False, return_context=True)
    print(rag_answer)
except Exception as e:
    print(f'ERROR! {e}')

=> Retrieved 1 docs

System: Using the information contained in the context,

            give a comprehensive answer to the question.

            Respond only to the question asked, response should be concise and relevant to the question.

            Provide the number of the source document when relevant.

            If the answer cannot be deduced from the context, do not give an answer.

Human: Context:

            ['Neurocognitive Disorders - Mild Neurocognitive Disorder : This sub-disease typically involves mild neurocognitive disorder\nKeywords: use, judgment, disorientation, memory, opioid, confusion, cognitive decline, mild, major, attention\n\nNeurocognitive Disorders - Opioid Use Disorder : This sub-disease typically involves opioid use disorder\nKeywords: use, judgment, disorientation, memory, opioid, confusion, cognitive decline, mild, major, attention\n\nNeurodevelopmental Disorders - Attention-Deficit/Hyperactivity Disorder (ADHD) : This sub-disease typically involve

In [42]:
QUESTION = 'I have a scenario for a disorder can say what is the sub type based on senario? the senario is: A 25-year-old woman hears voices that criticize her actions and believes she is being watched by government agents. She has withdrawn from friends and family.'

In [43]:
try:
    (rag_answer, rag_context) = classificationn_pipeline(QUESTION, reranker=False, return_context=True)
    print(rag_answer)
except Exception as e:
    print(f'ERROR! {e}')

=> Retrieved 1 docs

System: Using the information contained in the context,

            give a comprehensive answer to the question.

            Respond only to the question asked, response should be concise and relevant to the question.

            Provide the number of the source document when relevant.

            If the answer cannot be deduced from the context, do not give an answer.

Human: Context:

            ['Personality Disorders - Antisocial Personality Disorder : This sub-disease typically involves antisocial personality disorder\nKeywords: self-image, stability, relationships, impulsivity, borderline, antisocial, behavior, emotions, exhibitionistic\n\nPersonality Disorders - Borderline Personality Disorder : This sub-disease typically involves borderline personality disorder\nKeywords: self-image, stability, relationships, impulsivity, borderline, antisocial, behavior, emotions, exhibitionistic\n\nPersonality Disorders - Exhibitionistic Disorder : This sub-disease t

In [314]:
import re
import pandas as pd

# File path
file_path = '/kaggle/input/final-version/updated_scenarios_dataset.txt'

# Read the data from the file
with open(file_path, 'r', encoding='utf-8') as file:
    data = file.read()

# Regular expression to find scenarios, disorders, and categories
pattern = re.compile(r'\*\*Scenario\*\*: (.*?)\n\s+\*\*Disorder\*\*: (.*?)\n\s+\*\*Category\*\*: (.*?)\n', re.DOTALL)

# Find all matches
matches = pattern.findall(data)

# Create a list of dictionaries to store the parsed data
parsed_data = [{'Question': scenario.strip(), 'ground_truth': f"{disorder.strip()}  Category: {category.strip()}"} for scenario, disorder, category in matches]

# Convert the list of dictionaries to a pandas DataFrame
df = pd.DataFrame(parsed_data)

# Display the DataFrame
print(df)

# Save DataFrame to a CSV file for further analysis or usage
df.to_csv('parsed_mental_health_disorders.csv', index=False)

                                                                                                                                                                                     Question  \
0    A 6-year-old girl has difficulty making eye contact and often repeats phrases she hears from her favorite TV show. She becomes extremely distressed when her daily routine is disrupted.   
1                                                      An 8-year-old boy has limited interests, focusing only on trains. He struggles to interact with peers and prefers solitary activities.   
2                                                        A 4-year-old child does not respond to their name and avoids physical contact. They engage in repetitive hand flapping when excited.   
3                                                              A 7-year-old girl has significant delays in language development and prefers to play alone, often spinning objects repeatedly.   
4                                  

In [315]:
len(df)

217

In [316]:
question_list = df['Question'].to_list()
print(len(question_list))

217


In [340]:
from datasets import Dataset
import tqdm

RERANKER_MODEL_NAME = "colbert-ir/colbertv2.0"
reranker = RAGPretrainedModel.from_pretrained(RERANKER_MODEL_NAME)

def assemble_ragas_dataset(input_df):
    question_list = input_df['Question'].to_list()
    truth_list = input_df['ground_truth'].to_list()

    # Initialize the RAG pipeline to get the RAG answers and contexts
    rag_answer_list = []
    context_list = []


    for i in tqdm.tqdm(range(0, len(question_list))):
        question = question_list[i]
        truth = truth_list[i]
        
        # Get the RAG answer and contexts from the pipeline
        rag_answer, retrieved_contexts = rag_pipeline(question, reranker=reranker, return_context=True)

        # Extract the plain text from the retrieved Document objects
        retrieved_contexts_text = [doc.page_content for doc in retrieved_contexts]
        
        # Process the answer to extract everything after "Answer:"
        start_index = rag_answer.find("Answer:")
        if start_index != -1:
            final_answer = rag_answer[start_index:].strip()
        else:
            final_answer = rag_answer.strip()
        
        # Append the results to the respective lists
        rag_answer_list.append(final_answer)
        context_list.append(retrieved_contexts_text)  # This is now plain text

    # Create the dataset from the selected questions and answers
    ragas_ds = Dataset.from_dict({
        "question": question_list,  
        "contexts": context_list,
        "answer": rag_answer_list,
        "ground_truth": truth_list  
    })

    return ragas_ds

In [343]:
ragas_input_ds = assemble_ragas_dataset(df)

100%|██████████| 217/217 [1:13:14<00:00, 20.25s/it]


In [344]:
import json

# Specify the output file path
output_file = 'ragas_dataset.json'
output_file2 = 'ragas_dataset2.json'

# Open the file in write mode
# Initialize a list to hold the dataset entries
dataset_list = []
dataset_list2 = []

# Iterate over the dataset rows
for row in ragas_input_ds:
    # Extract the question, contexts, answer, and ground truth from the row
    entry = {
        "question": row['question'],
        "contexts": row['contexts'],  # This is a list of contexts
        "answer": row['answer'],
        "ground_truth": row['ground_truth']
    }
    
    # Append the entry to the list
    dataset_list.append(entry)

# Write the list of entries to a JSON file
with open(output_file, 'w', encoding='utf-8') as file:
    json.dump(dataset_list, file, indent=4, ensure_ascii=False)

print(f"Dataset has been successfully saved to {output_file}")

with open(output_file2, 'w', encoding='utf-8') as file:
    # Iterate over the dataset rows
    for row in ragas_input_ds:
    # Extract the question, contexts, answer, and ground truth from the row
        entry = {
            "question": row['question'],
            "answer": row['answer'],
            "ground_truth": row['ground_truth']
        }
    
        # Append the entry to the list
        dataset_list2.append(entry)

# Write the list of entries to a JSON file
with open(output_file2, 'w', encoding='utf-8') as file:
    json.dump(dataset_list2, file, indent=4, ensure_ascii=False)
    
print(f"Dataset has been successfully saved to {output_file2}")

Dataset has been successfully saved to ragas_dataset.json
Dataset has been successfully saved to ragas_dataset2.json


In [356]:

first_50_rows = ragas_input_ds.select(range(50))

# Filter even indices from these 50 rows
even_indices = [i for i in range(50) if i % 2 == 0]
subset = first_50_rows.select(even_indices)

# Display or use the subset
print(subset)

Dataset({
    features: ['question', 'contexts', 'answer', 'ground_truth'],
    num_rows: 25
})


In [358]:
import os

In [360]:
import os
import openai
import pprint
from openai import OpenAI
from ragas import evaluate
from ragas.metrics import context_precision, context_recall, faithfulness, answer_relevancy, summarization_score, context_precision, context_utilization, context_entity_recall
from ragas.llms import llm_factory
from langchain_openai.embeddings import OpenAIEmbeddings
from ragas.embeddings import LangchainEmbeddingsWrapper
from datasets import Dataset

# Save the API key in an environment variable.
openai_api_key = os.environ.get('OPENAI_API_KEY')

# Choose the metrics you want to see.
metrics = [context_precision, faithfulness, answer_relevancy,
        context_precision, context_utilization]

# Change the llm-as-critic to use a specific OpenAI model.
LLM_NAME = "gpt-4o-mini"
ragas_llm = llm_factory(model=LLM_NAME)

# Use the OpenAI embeddings for context comparison.
lc_embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
ragas_emb = LangchainEmbeddingsWrapper(embeddings=lc_embeddings)

# Assign the default models (LLM and embeddings) used for each metric.
for metric in metrics:
    metric.llm = ragas_llm
    metric.embeddings = ragas_emb

# Assuming the dataset has already been prepared using the 'assemble_ragas_dataset' function
# ragas_input_ds = assemble_ragas_dataset(eval_df)  # `eval_df` is prepared from the scenario file

# Evaluate the dataset directly
ragas_result = evaluate(
    subset,
    metrics=metrics,
    llm=ragas_llm,
)

# Convert the results to a Pandas DataFrame for viewing.
ragas_output_df = ragas_result.to_pandas()
print(ragas_output_df)

Evaluating:   0%|          | 0/125 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
print(ragas_output_df)

In [365]:
import getpass 
import os 
from langchain_openai import ChatOpenAI 
from langchain_core.messages import HumanMessage, SystemMessage 
from langchain_core.output_parsers import StrOutputParser 
from langchain_core.prompts import ChatPromptTemplate 
from langchain_core.prompts import PromptTemplate 
 
 
def translate(input_text,source_lang,dest_lang) :  
   
  parser = StrOutputParser() 
  model = ChatOpenAI(model="gpt-4o-mini" , temperature = 0) 
 
  translate_template = "Translate the following {source} text to {dest} : {text} . only give translated language" 
  translate_prompt = PromptTemplate.from_template(translate_template) 
 
  chain = translate_prompt | model | parser 
  result = chain.invoke({"source":f"{source_lang}","dest":f"{dest_lang}","text": f"{input_text}"}) 
 
  return result
RERANKER_MODEL_NAME = "colbert-ir/colbertv2.0"
reranker = RAGPretrainedModel.from_pretrained(RERANKER_MODEL_NAME)
que = "A 10-year-old boy has difficulty staying seated in class and frequently interrupts others. He often forgets to complete homework assignments"
try:
    rag_answer = rag_pipeline(que, reranker=reranker, return_context=False)
    start_index = rag_answer.find("Answer:")

    # If "Answer:" is found, print everything after that
    if start_index != -1:
        print(translate(rag_answer[start_index:].strip(),"english","persian"))
    else:
        print(translate(rag_answer),"english","persian")
except Exception as e:
    print(f'ERROR! {e}')

100%|██████████| 1/1 [00:00<00:00, 31.11it/s]


پاسخ: نام اختلال: اختلال کمبود توجه/بیش فعالی (ADHD) دسته: اختلالات نورودولتی
توضیح: علائم پسر، از جمله مشکل در نشستن، قطع کردن صحبت دیگران، فراموش کردن تکالیف، مشکل در پیروی از دستورالعمل‌ها، مشکل در انتظار نوبت خود و گم کردن اشیاء، با معیارهای تشخیصی ADHD همخوانی دارد.


In [364]:
rag_answer

'System: You are a licensed psychologist with extensive expertise in diagnosing psychological disorders.\n    Your task is to analyze the provided scenario and determine the most likely psychological disorder based solely on the data given to you.\n\n    Important Instructions:\n    1. You must identify the disorder in this strict format:\n       Disorder Name Category: Broader Category Name\n       (e.g., Autism Spectrum Disorder Category: Neurodevelopmental Disorders)\n    2. After identifying the disorder, provide a clear and well-reasoned explanation for why this specific disorder was diagnosed, strictly using the information from the scenario. All reasoning must be rooted in the data presented.\n    3. It is forbidden to provide an answer in any format other than the specified structure.\n    4. You must always provide an answer based on the data given in the scenario, even if the information seems limited or ambiguous. Stick to the data and do not introduce external assumptions.\